In [ ]:
## Reading the file

import os
import numpy as np
import matplotlib.pyplot as plt

ts = np.load("sod_shock_tube_1_analytical.npz", allow_pickle=True)["arr_0"].item()

i = 0
for time in ts.keys():
    data = ts[time]
    r = data[:,0]
    p = data[:,1]
    rho = data[:,2]
    vel = data[:,3]
    eps = data[:,4]
    W = 1/np.sqrt(1-vel**2)
    D = rho * W

    plt.plot(r, rho)
    plt.savefig(f"frames/frame.{i:04d}.png")
    plt.close('all')

    i += 1

os.system("~/perspective/create_video.py --frame-rate=15 --frames-dir frames --output sod_shock_tube_1_analytical_rho.mp4")
os.system(f'rm frames/*.png')


In [ ]:
# Create the file (requires a modified riemann_f90 binary)
import subprocess

times = np.linspace(0, 0.5, 400)

solution_timeseries = {}

for t in times:
    try:
        os.system("rm solution.dat")
    except:
        pass

    rd = {
        "gamma": 5/3,
        "t": t,
        "p_left": 40/3,
        "rho_left": 10,
        "v_left": 0,
        "p_right": 0,
        "rho_right": 1,
        "v_right": 0,
    }

    p = subprocess.run(['./riemann_f90'],
                    input=f'{rd["gamma"]}\n{rd["t"]}\n{rd["p_left"]}\n{rd["rho_left"]}\n{rd["v_left"]}\n{rd["p_right"]}\n{rd["rho_right"]}\n{rd["v_right"]}\n',
                    capture_output=True, text=True)

    print(p.stdout)

    data = np.loadtxt('solution.dat')
    solution_timeseries[t] = data


np.savez_compressed("sod_shock_tube_1_analytical2", solution_timeseries)